#Newcon Import Tables

##Creating Variables

In [0]:
# dbutils.widgets.text('Catalogo','')
# dbutils.widgets.text('CatalogoClone','')
# dbutils.widgets.text('Database','')
# dbutils.widgets.text('TableName','')
# dbutils.widgets.text('KafkaServer','')
# dbutils.widgets.text('KafkaUsername','')
# dbutils.widgets.text('KafkaPassword','')
# dbutils.widgets.text('CheckpointLocation','')
# dbutils.widgets.text('TableLocation','')
# dbutils.widgets.text('TempLocation','')

##Setting Variables

In [0]:
Catalogo = dbutils.widgets.get("Catalogo")
CatalogoClone = dbutils.widgets.get("CatalogoClone")
Database = dbutils.widgets.get("Database")
TableName = dbutils.widgets.get("TableName")

CheckpointLocation = dbutils.widgets.get("CheckpointLocation")
TableLocation = dbutils.widgets.get("TableLocation")
TempLocation = dbutils.widgets.get("TempLocation")

KafkaServer = dbutils.widgets.get("KafkaServer")
KafkaUsername = dbutils.widgets.get("KafkaUsername")
KafkaPassword = dbutils.widgets.get("KafkaPassword")
KafkaTopic = f"src-cdc-sql-newcon-embracon-{TableName}-json"

TableName = TableName.lower()

##Library Imports

In [0]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import *

#Bronze Layer

##Create Ingestion Fuction

In [0]:
def KafkaIngestion(TableName, Schema):
    Raw = (
        spark.readStream.format("kafka")
        .option("kafka.bootstrap.servers", KafkaServer)
        .option("kafka.sasl.mechanism", "PLAIN")
        .option("kafka.security.protocol", "SASL_SSL")
        .option("kafka.client.dns.lookup", "use_all_dns_ips")
        .option("kafka.basic.auth.credentials.source", "USER_INFO")
        .option(
            "kafka.sasl.jaas.config",
            f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="{KafkaUsername}" password="{KafkaPassword}";',
        )
        .option("subscribe", KafkaTopic)
        .option("startingOffsets", "earliest")
        .load()
    )

    Bronze = Raw.select(
        col("key").cast("string").alias("eventId"),
        from_json(col("value").cast("string"), Schema).alias("json"),
    ).select("json.after.*", "json.op", "json.ts_ms")

    Bronze.writeStream.outputMode("append").option(
        "checkpointLocation", f"{CheckpointLocation}/bronze/{TableName}"
    ).trigger(availableNow=True).table(f"{Catalogo}.{Database}.{TableName}")

    spark.sql(
        f"""CREATE TABLE {CatalogoClone}.{Database}.{TableName} SHALLOW CLONE {Catalogo}.{Database}.{TableName}"""
    )

##Call Ingestion Function

In [0]:
KafkaIngestion(TableName,
               StructType([StructField('after', StructType([StructField('CD_Base_Origem', StringType(), True), StructField('CD_Cota', LongType(), True), StructField('CD_Cota_Origem', LongType(), True), StructField('CD_Grupo', StringType(), True), StructField('CD_Grupo_Origem', LongType(), True), StructField('ID_Cota', LongType(), True), StructField('Versao', LongType(), True), StructField('Versao_Origem', LongType(), True)]), True), StructField('before', StringType(), True), StructField('op', StringType(), True), StructField('source', StructType([StructField('change_lsn', StringType(), True), StructField('commit_lsn', StringType(), True), StructField('connector', StringType(), True), StructField('db', StringType(), True), StructField('event_serial_no', StringType(), True), StructField('name', StringType(), True), StructField('schema', StringType(), True), StructField('sequence', StringType(), True), StructField('snapshot', StringType(), True), StructField('table', StringType(), True), StructField('ts_ms', LongType(), True), StructField('version', StringType(), True)]), True), StructField('transaction', StringType(), True), StructField('ts_ms', LongType(), True)]))

##Data Ingestion

In [0]:
# from pyspark.sql.functions import col, from_json

# Raw = (
#     spark.readStream.format("kafka")
#     .option("kafka.bootstrap.servers", KafkaServer)
#     .option("kafka.sasl.mechanism", "PLAIN")
#     .option("kafka.security.protocol", "SASL_SSL")
#     .option("kafka.client.dns.lookup", "use_all_dns_ips")
#     .option("kafka.basic.auth.credentials.source", "USER_INFO")
#     .option(
#         "kafka.sasl.jaas.config",
#         f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="{KafkaUsername}" password="{KafkaPassword}";',
#     )
#     .option("subscribe", KafkaTopic)
#     .option("startingOffsets", "earliest")
#     .load()
# )

In [0]:
# dbutils.fs.mkdirs('/dados')

In [0]:
# dbutils.fs.put("/dados/conco060.json", '{"before":null,"after":{"ID_Advogado":1,"CD_Advogado":"000001","NM_Fantasia":"J.F. Silva Adv. Associados","ID_Pessoa":1311761,"ID_Endereco":4979719,"ID_Telefone":2648599,"ID_Fax":null,"ID_E_Mail":null},"source":{"version":"2.4.2.Final","connector":"sqlserver","name":"src-cdc-sql-newcon-embracon","ts_ms":1714447429448,"snapshot":"first_in_data_collection","db":"Newcon_Embracon","sequence":null,"schema":"dbo","table":"CONCO060","change_lsn":null,"commit_lsn":"000e5d5e:0003d608:0004","event_serial_no":null},"op":"r","ts_ms":1714447429436,"transaction":null}', True)

In [0]:
# Schema = spark.read.json(f"/dados/conco060.json").schema

In [0]:
# Bronze = Raw.select(
#     col("key").cast("string").alias("eventId"),
#     from_json(col("value").cast("string"), Schema).alias("json"),
# ).select("json.after.*", "json.op", "json.ts_ms")

# # display(bronzedf)

In [0]:
# Bronze.writeStream.outputMode("append").option(
#     "checkpointLocation", f"{CheckpointLocation}/bronze/{TableName}"
# ).trigger(availableNow=True).table(f"{Database}.{TableName}")

In [0]:
# spark.sql(f"""CREATE TABLE {DatabaseClone}.{TableName} SHALLOW CLONE {Database}.{TableName}""")

In [0]:
# %sql
# drop table embracon_des.newcon_bronze.conco060

In [0]:
# %sql
# CREATE TABLE embracon_des.newcon_bronze.conco060 SHALLOW CLONE embracon.newcon_bronze.conco060

#Silver Layer

In [0]:
# silverDF = spark.readStream.format("delta") \
#   .table("events_bronze") \
#   .select("eventId", "json.*")

# display(silverDF)

In [0]:
# silverDF.writeStream.format("delta") \
#   .outputMode("append") \
#   .option("checkpointLocation", f"{project_dir}/checkpoints/silver") \
#   .table("events_silver")

In [0]:
# %sql
# SELECT action, count(1) FROM events_silver GROUP BY action

#Schema CONCO060

In [0]:
# from pyspark.sql.types import *

In [0]:
# schema = StructType([StructField('after', StructType([StructField('CD_Advogado', StringType(), True), StructField('ID_Advogado', LongType(), True), StructField('ID_E_Mail', StringType(), True), StructField('ID_Endereco', LongType(), True), StructField('ID_Fax', StringType(), True), StructField('ID_Pessoa', LongType(), True), StructField('ID_Telefone', LongType(), True), StructField('NM_Fantasia', StringType(), True)]), True), StructField('before', StringType(), True), StructField('op', StringType(), True), StructField('source', StructType([StructField('change_lsn', StringType(), True), StructField('commit_lsn', StringType(), True), StructField('connector', StringType(), True), StructField('db', StringType(), True), StructField('event_serial_no', StringType(), True), StructField('name', StringType(), True), StructField('schema', StringType(), True), StructField('sequence', StringType(), True), StructField('snapshot', StringType(), True), StructField('table', StringType(), True), StructField('ts_ms', LongType(), True), StructField('version', StringType(), True)]), True), StructField('transaction', StringType(), True), StructField('ts_ms', LongType(), True)])
